In [1]:
import sys
sys.path.append('/home/mila/l/leo.gagnon/latent_control')

%load_ext autoreload
%autoreload 2
from lightning_modules.diffusion_prior import DiffusionPriorTask
import torch
import matplotlib.pyplot as plt
from data.diffusion import KnownLatentDiffusionDataset
from torch2jax import j2t, t2j

/home/mila/l/leo.gagnon/latent_control/venv/lib/python3.10/site-packages/pl_bolts/__init__.py:11: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(numpy, tp_name):
/home/mila/l/leo.gagnon/latent_control/venv/lib/python3.10/site-packages/pl_bolts/__init__.py:11: FutureWarning: In the future `np.bool` will be defined as the corresponding NumPy scalar.
  if not hasattr(numpy, tp_name):
/home/mila/l/leo.gagnon/latent_control/venv/lib/python3.10/site-packages/pl_bolts/models/self_supervised/amdim/amdim_module.py:34: UnderReviewWarning: The feature generate_power_seq is currently marked under review. The compatibility with other Lightning projects is not guaranteed and API may change at any time. The API and functionality may change without warning in future releases. More details: https://lightning-bolts.readthedocs.io/en/latest/stability.html
  "lr_options": generate_power_seq(LEARNING_RATE_CIFAR, 11),
/home/mila/l/leo.gagnon/late

In [4]:
task = DiffusionPriorTask('61xaxbsl')
task.cuda();

Loaded dataset : (11288/1000)
Loaded checkpoing : last.ckpt
Loaded checkpoing : last.ckpt


In [5]:
dataset = task.train_data.dataset 
dataset: KnownLatentDiffusionDataset

In [14]:
latent, cond_input_ids, cond_tokens, cond_ignore_mask = dataset[3121].values()

/home/mila/l/leo.gagnon/latent_control/data/hmm.py:585: FutureWarning: None encountered in jnp.array(); this is currently treated as NaN. In the future this will result in an error.
  intv_envs = jnp.array(intv_envs)


In [15]:
cond_mask = torch.BoolTensor([True]*200)[None].cuda()

In [28]:
task.diffusion_prior.cfg.sampler = 'dpmpp' # ddpm, dpmpp

In [29]:
z_t = task.diffusion_prior.sample(100, cond_tokens=cond_tokens, cond_mask=cond_mask, cls_free_guidance=1.0)
#z_t = task.diffusion_prior.unnormalize_latent(z_t)

sampling loop time step:   0%|          | 0/250 [00:00<?, ?it/s]

In [30]:
z_t = task.diffusion_prior.unnormalize_latent(z_t)

In [31]:
[torch.Tensor([(latent_embds.weight @ z_t[i].T).argmax() for latent_embds in task.base_task.model.encoder.latent_embedding]) for i in range(50)]

[tensor([0., 1., 1., 0., 1., 0., 0., 0., 0., 0., 0., 2.]),
 tensor([1., 1., 1., 0., 1., 0., 1., 0., 0., 0., 0., 2.]),
 tensor([1., 1., 1., 0., 1., 0., 0., 1., 0., 0., 0., 2.]),
 tensor([3., 1., 1., 0., 0., 0., 1., 1., 0., 0., 0., 2.]),
 tensor([0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 2.]),
 tensor([3., 1., 1., 0., 0., 0., 1., 1., 0., 0., 0., 0.]),
 tensor([2., 1., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0.]),
 tensor([1., 1., 1., 0., 1., 0., 1., 0., 0., 0., 0., 2.]),
 tensor([3., 0., 1., 1., 0., 0., 1., 1., 0., 0., 0., 0.]),
 tensor([2., 1., 0., 0., 1., 0., 1., 1., 0., 0., 0., 2.]),
 tensor([2., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 2.]),
 tensor([3., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 tensor([0., 0., 1., 0., 0., 0., 1., 1., 0., 0., 0., 1.]),
 tensor([2., 1., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0.]),
 tensor([0., 0., 1., 0., 1., 0., 1., 1., 0., 0., 0., 2.]),
 tensor([1., 0., 1., 0., 1., 1., 1., 1., 0., 0., 0., 2.]),
 tensor([0., 1., 1., 0., 1., 0., 0., 1., 0., 0., 0., 2.]

In [22]:
env_latents = j2t(task.base_task.full_data.index_to_latent)[0].to(torch.long).cuda()[None]
env_latents = task.base_task.model.encoder(true_latents=env_latents)
env_latents

tensor([[[ 1.5561e+00,  7.4856e-01,  4.1153e+00, -2.4731e+00, -1.4466e+00,
          -1.2991e+00,  1.7986e+00, -4.4539e+00,  4.6276e+00, -1.2840e+00,
           2.8628e+00, -3.8792e+00, -5.9071e-01, -3.0843e+00, -1.9485e+00,
          -5.2641e+00, -2.8646e+00, -7.0337e-01, -1.9427e+00, -4.3665e-02,
          -3.2926e+00,  7.2712e+00, -2.4647e+00, -3.3398e-01, -9.0019e-01,
           2.2383e+00, -1.4827e+00, -5.2373e+00, -1.0177e+00,  8.3677e-01,
           1.1302e+00,  2.2745e-01, -8.5907e-01, -9.6679e-02,  1.4911e+00,
          -2.5468e+00, -2.5649e-01,  1.3726e+00,  1.0484e+00,  5.3324e+00,
           2.2521e+00, -2.0496e-01, -2.1855e+00,  9.1902e-01,  1.8415e+00,
          -8.1387e-02, -3.7777e-01, -5.3471e+00,  1.3217e+00, -1.6922e+00,
          -8.1620e-02,  1.1876e+00, -3.2295e+00,  4.5442e-01, -2.4309e+00,
          -2.0893e+00, -1.1472e-01,  7.5290e-01, -6.6094e-01, -5.4295e+00,
          -1.5896e+00, -8.8771e-01,  1.8211e+00, -2.7817e-01, -3.2782e+00,
           4.9800e-01,  1

In [95]:
task.base_task.model.encoder.latent_embedding[3].weight @ z_t[0].T

tensor([[ 179.3701],
        [-104.5878]], device='cuda:0')

In [19]:
task.base_task.model.encoder.latent_embedding[1].weight @ z_t[0].T

tensor([[189.8581],
        [ -6.5179]], device='cuda:0')

In [31]:
task.base_task.full_data.index_to_latent[0]

Array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int16)

In [139]:
[torch.Tensor([(latent_embds.weight @ z_t[i].T).argmax() for latent_embds in task.base_task.model.encoder.latent_embedding]) for i in range(10)]

[tensor([2., 1., 1., 0., 1., 0., 0., 1., 0., 0., 0., 2.]),
 tensor([2., 0., 1., 1., 1., 0., 0., 1., 0., 0., 0., 2.]),
 tensor([3., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.]),
 tensor([3., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0.]),
 tensor([3., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 2.]),
 tensor([0., 1., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0.]),
 tensor([3., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 2.]),
 tensor([1., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 2.]),
 tensor([0., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0.]),
 tensor([1., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.])]